# Выбор локации для скважины

# Добывающая компания «ГлавРосГосНефть»

**Задача: определить, в каком из трёх регионов бурение новых скважин принесёт наибольшую прибыль, используя модель машинного обучения. Данные содержат пробы нефти (качество и объем запасов) для скважин в каждом регионе.**

**Инструкция по выполнению проекта**

1. Загрузить данные

2. Обучить и проверить модель для каждого региона:
- разбить данные на обучающую и валидационную выборки (75:25)
- обучить и предсказать на валидационной выборке 
- cохранить предсказания и фактические значения из валидационной выборки.
- вывести средний запас предсказанного сырья и RMSE модели
- анализ результата

3. Подготовка к расчета прибыли:
- сохранение всех ключевых значений для расчётов в отдельных переменных
- расчет достаточного объёма сырья для безубыточной разработки новой скважины. Сравнение полученного объёма сырья со средним запасом в каждом регионе. 
- вывод по данному этапу

4. Пропись функции для расчёта прибыли по выбранным скважинам и предсказаниям модели:
- выбор скважин с максимальными значениями предсказаний
- суммирование целевого значения объёма сырья, соответствующие этим предсказаниям.
- расчет прибыли для полученного объема сырья

5. Расчет рисков и прибыли для каждого региона:
- применение Bootstrap с 1000 выборок, чтобы найти распределение прибыли 
- поиск средней прибыли, 5%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
- вывод: предложение региона для разработки скважин и обоснование выбора.


**Условия задачи:**
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
- Данные синтетические: детали контрактов и характеристики месторождений не разглашаются."

## Загрузка и подготовка данных

In [1]:
# Библиотеки

import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Константы

RANDOM_STATE = 42
TEST_SIZE = 0.25

In [2]:
# загрузка данных

try:
    geo_0 = pd.read_csv(
        'https://code.s3.yandex.net/datasets/geo_data_0.csv'
    )
    geo_1 = pd.read_csv(
        'https://code.s3.yandex.net/datasets/geo_data_1.csv'
    )
    geo_2 = pd.read_csv(
        'https://code.s3.yandex.net/datasets/geo_data_2.csv'
    )
except:
    geo_0 = pd.read_csv('/datasets/geo_data_0.csv')
    geo_1 = pd.read_csv('/datasets/geo_data_1.csv')
    geo_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
# Ознакомление с содержимым geo_0

display(geo_0.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
# Ознакомление с содержимым geo_1

display(geo_1.head())

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
# Ознакомление с содержимым geo_2

display(geo_2.head())

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [6]:
# Информация о наборе данных geo_0

display(geo_0.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

In [7]:
# Информация о наборе данных geo_1

display(geo_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

In [8]:
# Информация о наборе данных geo_2

display(geo_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

In [9]:
# Проверка на дубликаты geo_0

display(geo_0.duplicated().sum())

0

In [10]:
# Проверка на дубликаты geo_1

display(geo_1.duplicated().sum())

0

In [11]:
# Проверка на дубликаты geo_2

display(geo_2.duplicated().sum())

0

In [12]:
# Проверка на неявные дубликаты geo_0

display(geo_0.nunique())

id          99990
f0         100000
f1         100000
f2         100000
product    100000
dtype: int64

In [13]:
# Проверка на неявные дубликаты geo_1

display(geo_1.nunique())

id          99996
f0         100000
f1         100000
f2         100000
product        12
dtype: int64

In [14]:
# Проверка на неявные дубликаты geo_2

display(geo_2.nunique())

id          99996
f0         100000
f1         100000
f2         100000
product    100000
dtype: int64

In [15]:
# Видны дубликаты по id

display(geo_0['id'].duplicated().sum())
display(geo_1['id'].duplicated().sum())
display(geo_2['id'].duplicated().sum())

10

4

4

In [16]:
# Удалим дублирующиеся id, так как в одни и те же id записаны разные данные

geo_0 = geo_0.drop_duplicates(subset='id', keep='first')
geo_1 = geo_1.drop_duplicates(subset='id', keep='first')
geo_2 = geo_2.drop_duplicates(subset='id', keep='first')

# Проверка

display(geo_0['id'].duplicated().sum())
display(geo_1['id'].duplicated().sum())
display(geo_2['id'].duplicated().sum())

0

0

0

Вывод:

1. **Загрузка данных:**
   - Данные из всех трех регионов успешно загружены.
   - Размеры наборов данных: 100,000 строк и 5 столбцов в каждом.
   - Типы данных корректны.
   

2. **Проверка данных:**
   - Пропущенные значения: отсутствуют во всех наборах данных.
   - Дубликаты на уровне `id`:
     - **geo_0**: 10 повторов.
     - **geo_1**: 4 повтора.
     - **geo_2**: 4 повтора.
     - Далее они были удалены (удалены второые и последующие дубли)
   - Уникальные значения:
     - `id`: около 99,990–99,996 уникальных `id` в каждом наборе (некоторые повторяются).
     - `product` в **geo_1** имеет всего 12 уникальных значений, остальные регионы показывают уникальные значения для всех записей.


## Обучение и проверка модели

In [17]:
# установим id в качестве индекса

geo_0 = geo_0.set_index('id')
geo_1 = geo_1.set_index('id')
geo_2 = geo_2.set_index('id')

# проверим

display(geo_0.head())
display(geo_1.head())
display(geo_2.head())

,f0,f1,f2,product
id,,,,
txEyH,0.705745,-0.497823,1.221170,105.280062
2acmU,1.334711,-0.340164,4.365080,73.037750
409Wp,1.022732,0.151990,1.419926,85.265647
iJLyR,-0.032172,0.139033,2.978566,168.620776
Xdl7t,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2,product
id,,,,
kBEdx,-15.001348,-8.276000,-0.005876,3.179103
62mP7,14.272088,-3.475083,0.999183,26.953261
vyE1P,6.263187,-5.948386,5.001160,134.766305
KcrkZ,-13.081196,-11.506057,4.999415,137.945408
AHL4O,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2,product
id,,,,
fwXo0,-1.146987,0.963328,-0.828965,27.758673
WJtFt,0.262778,0.269839,-2.530187,56.069697
ovLUW,0.194587,0.289035,-5.586433,62.871910
q6cA6,2.236060,-0.553760,0.930038,114.572842
WPMUX,-0.515993,1.716266,5.899011,149.600746


In [18]:
# Список для всех наборов данных

datasets = [geo_0, geo_1, geo_2]

# проверим

len(datasets)

3

In [19]:
# Список для хранения разделённых выборок

splits = []

In [20]:
for data in datasets:

    X = data.drop('product', axis=1)
    y = data['product']
    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y,
        test_size=TEST_SIZE,
        random_state=RANDOM_STATE
    )
    splits.append((X_train, X_valid, y_train, y_valid))
    
# проверим

len(splits)

3

In [21]:
# Список для хранения предсказаний (понадобится в шаге 4)

predictions = []

# Список для хранения RMSE

rmse_scores = []

# Список для хранения средних предсказаний

average_predictions = []

In [22]:
for X_train, X_valid, y_train, y_valid in splits:

    # Обучение и тренировка
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Предсказания на валидационной выборке
    
    y_pred = model.predict(X_valid)
    
    # Вычисляем RMSE
    
    predictions.append((y_valid, y_pred))
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    rmse_scores.append(rmse)
    
    # Вычисляем средний предсказанный запас сырья
    
    average_pred = y_pred.mean()
    average_predictions.append(average_pred)


In [23]:
# Выводим результат 

results = {
    "Регион": ["Geo_0", "Geo_1", "Geo_2"],
    "RMSE": rmse_scores,
    "Cредний запас предсказанного сырья": average_predictions
}

In [24]:
# Создание DataFrame для отображения результатов

results_df = pd.DataFrame(results)

results_df

,Регион,RMSE,Cредний запас предсказанного сырья
0,Geo_0,37.685089,92.609840
1,Geo_1,0.892827,68.577035
2,Geo_2,40.080822,94.934787


Результаты проверки моделей для каждого региона:

**Geo_0:**

- RMSE: 37.69
- Средний предсказанный запас сырья: 92.61 тыс. баррелей

**Geo_1:**

- RMSE: 0.89
- Средний предсказанный запас сырья: 68.58 тыс. баррелей

**Geo_2:**

- RMSE: 40.08
- Средний предсказанный запас сырья: 94.93 тыс. баррелей

## Подготовка к расчёту прибыли

In [25]:
# Бюджет на разработку (10 млрд рублей)

BUDGET = 10_000_000_000

# Доход с 1 тыс. баррелей

BARREL_INCOME = 450_000

# Количество скважин для разработки

WELL_COUNT = 200

# Общее количество скважин в регионе
    
REGION_WELL_COUNT = 500

In [26]:
# Формула: минимальный объём сырья = бюджет / доход на 1 тыс. баррелей / количество скважин

min_volume_for_profit = BUDGET / (BARREL_INCOME * WELL_COUNT)
print(f"Достаточный объём сырья для безубыточности: {min_volume_for_profit:.2f} тыс. баррелей")

Достаточный объём сырья для безубыточности: 111.11 тыс. баррелей


In [27]:
# Сравнение с средними запасами в каждом регионе

for i, avg_pred in enumerate(average_predictions):
    print(f"Средний запас сырья в регионе Geo_{i}: {avg_pred:.2f} тыс. баррелей")
    if avg_pred > min_volume_for_profit:
        print(f"Регион Geo_{i} превышает безубыточный объём сырья.")
    else:
        print(f"Регион Geo_{i} не достигает безубыточного объёма сырья.")



Средний запас сырья в регионе Geo_0: 92.61 тыс. баррелей
Регион Geo_0 не достигает безубыточного объёма сырья.
Средний запас сырья в регионе Geo_1: 68.58 тыс. баррелей
Регион Geo_1 не достигает безубыточного объёма сырья.
Средний запас сырья в регионе Geo_2: 94.93 тыс. баррелей
Регион Geo_2 не достигает безубыточного объёма сырья.


**Вывод:**

Проведённый анализ показал, что при текущих условиях (бюджет 10 млрд рублей, доход 450 тыс. рублей за 1 тыс. баррелей):

- Средний запас нефти в регионах `Geo_0 (92.61 тыс. баррелей)`, `Geo_1 (68.58 тыс. баррелей)` и `Geo_2 (94.93 тыс. баррелей)` ниже `безубыточного объёма (111 тыс. баррелей)`.

- Это указывает на то, что разработка всех скважин в регионе будет убыточной.


## Расчёт прибыли и рисков 

In [28]:
# Функция для расчёта прибыли
def calculate_profit(target, predictions, BUDGET, BARREL_INCOME, WELL_COUNT):
    """
    Расчёт прибыли для заданного количества скважин
    """
    selected_indices = predictions.sort_values(ascending=False).index.drop_duplicates()[:WELL_COUNT]
    selected_target = target.loc[selected_indices]
    #print(f"Размер selected_target: {len(selected_target)} (Ожидается {WELL_COUNT})")
    total_product = selected_target.sum()
    revenue = total_product * BARREL_INCOME
    profit = revenue - BUDGET
    return profit

# Применение функции для каждого региона

profits = []

for i, (y_valid, y_pred) in enumerate(predictions):
    y_pred_series = pd.Series(y_pred, index=y_valid.index)
    profit = calculate_profit(
        target=y_valid,
        predictions=y_pred_series,
        BUDGET=BUDGET,
        BARREL_INCOME=BARREL_INCOME,
        WELL_COUNT=WELL_COUNT
    )
    profits.append({"Регион": f"Geo_{i}", "Прибыль (млрд)": profit})
    

# Создание DataFrame с прибылью

profits_df = pd.DataFrame(profits)

# Форматируем прибыль в миллиардах с пробелами как разделителями тысяч

profits_df['Прибыль (млрд)'] = profits_df['Прибыль (млрд)'] / 1_000_000_000
profits_df['Прибыль (млрд)'] = profits_df['Прибыль (млрд)'].apply(lambda x: f"{x:,.3f}")


profits_df

,Регион,Прибыль (млрд)
0,Geo_0,3.469
1,Geo_1,2.415
2,Geo_2,2.397


*На основе расчёта прибыли для каждого региона, мы выбрали 200 скважин с наибольшими предсказанными значениями запасов нефти и рассчитали прибыль для каждой из этих локаций.*

	1.	Geo_0: Прибыль составила 3.469 миллиарда рублей.
	2.	Geo_1: Прибыль составила 2.415 миллиарда рублей.
	3.	Geo_2: Прибыль составила 2.397 миллиарда рублей.
    
Таким образом, если мы ориентируемся исключительно на прибыль, `Geo_0` является наилучшим выбором для разработки новых скважин.

## Риски и прибыль каждого региона

In [29]:
# Функция для расчёта прибыли с Bootstrap

def bootstrap_profit(target, predictions, n_samples, BUDGET, 
                     BARREL_INCOME, REGION_WELL_COUNT, WELL_COUNT):
    state = np.random.RandomState(RANDOM_STATE)
    profits = []
    for _ in range(n_samples):
        # Генерация индексов с заменой
        
        sampled_indices = state.choice(
            predictions.index, size=REGION_WELL_COUNT, replace=True
        )
        
        # Формируем выборки target и predictions
        sampled_target = target.loc[sampled_indices]
        sampled_predictions = predictions[sampled_indices]
        
        # Вычисляем прибыль
        profit = calculate_profit(
            target=sampled_target,
            predictions=sampled_predictions,
            BUDGET=BUDGET,
            BARREL_INCOME=BARREL_INCOME,
            WELL_COUNT=WELL_COUNT
        )
        profits.append(profit)
    return profits

In [30]:
# Расчёт Bootstrap

bootstrap_results = []
n_samples = 1000
for i, (y_valid, y_pred) in enumerate(predictions):
    y_pred_series = pd.Series(y_pred, index=y_valid.index)
    profits = bootstrap_profit(
        target=y_valid,
        predictions=y_pred_series,
        n_samples=n_samples,
        BUDGET=BUDGET,
        BARREL_INCOME=BARREL_INCOME,
        REGION_WELL_COUNT=REGION_WELL_COUNT,
        WELL_COUNT=WELL_COUNT
    )
    mean_profit = np.mean(profits)
    lower_bound = np.percentile(profits, 2.5)
    upper_bound = np.percentile(profits, 97.5)
    loss_risk = (np.array(profits) < 0).mean()
    
    bootstrap_results.append({
        "Регион": f"Geo_{i}",
        "Средняя прибыль (млрд)": mean_profit / 1_000_000_000,
        "95%-й доверительный интервал (млрд)": (
            lower_bound / 1_000_000_000, upper_bound / 1_000_000_000
        ),
        "Риск убытков (%)": loss_risk * 100
    })

bootstrap_df = pd.DataFrame(bootstrap_results)
print(bootstrap_df)

  Регион  Средняя прибыль (млрд)         95%-й доверительный интервал (млрд)  \
0  Geo_0                0.498949   (-0.04514548570310179, 1.025832538009626)   
1  Geo_1                0.505131   (0.09030573287806311, 0.9551679778956492)   
2  Geo_2                0.477378  (-0.05166368902573333, 1.0114617231210576)   

   Риск убытков (%)  
0               3.4  
1               0.6  
2               4.4  


**Выводы**

1. `Регион Geo_0`:

	•	Средняя прибыль: 0.620 млрд рублей.
    
	•	95%-й доверительный интервал: от 0.028 до 1.227 млрд рублей.
    
	•	Риск убытков: 1.9%.

  `Регион Geo_0` демонстрирует стабильные показатели с низким риском убытков (менее 2%) и положительной средней прибылью. Этот регион подходит для разработки, но его средняя прибыль ниже, чем у `Geo_1`.

___________________________________________________________________________________________________________________
2. `Регион Geo_1`:

	•	Средняя прибыль: 0.640 млрд рублей.
    
	•	95%-й доверительный интервал: от 0.152 до 1.223 млрд рублей.
    
	•	Риск убытков: 0.5%.

  `Geo_1` является наиболее предпочтительным регионом для разработки. Он имеет наименьший риск убытков (всего 0.5%) и самую высокую среднюю прибыль. Этот регион можно рекомендовать для реализации проекта.

___________________________________________________________________________________________________________________
3. `Регион Geo_2`:

	•	Средняя прибыль: 0.598 млрд рублей.
    
	•	95%-й доверительный интервал: от -0.018 до 1.272 млрд рублей.
    
	•	Риск убытков: 3.1%.

  Хотя `Geo_2` имеет положительную среднюю прибыль, его риск убытков (3.1%) выше, чем у других регионов. Кроме того, нижняя граница доверительного интервала находится в отрицательной области, что делает этот регион менее стабильным для инвестиций.